The default settings in Bilbac ccincide with the 'aflow' settings, except for Bilbao = hex, aflow = rhomb

In [ ]:
# Scrape the unique translation vectors found in the general positions


import requests
import html2text
from sympy import parse_expr,symbols
import json
x,y,z,r,s,t = symbols("x y z r s t")

translation_parts={}

for i in range(230):
    unique_translations = set()
    sgnum = i+1        
    data = html2text.html2text(requests.get(f"https://www.cryst.ehu.es/cgi-bin/cryst/programs/nph-getgen?what=text&gnum={sgnum}").text)
    for line in data.splitlines():
        line_split = line.split()
        try:
            assert(str(int(line_split[0])) == line_split[0])
        except Exception as e:
            continue
        operations = [parse_expr(coord) for coord in line_split[1].split(',')]
        translations = [float(op.evalf(subs={x: 0, y: 0, z: 0, r: 0, s: 0, t: 0})) for op in operations]
        unique_translations.add(tuple(translations))
    translation_parts[sgnum] = sorted(list(unique_translations))
    
with open('translation_parts.json','w') as f:
    json.dump(translation_parts,f)


In [ ]:
# Scrape the unique translation vectors found in the coset representatives of the space group in its normalizer.
# Note that, at least for SG#9 and possibly some other monoclinic SGs, you need to manually add translations from the affine normalizer, as they are not 
# represented in the WYCKSETS program (for SG#15 they are, as a special exception)

import pandas as pd
from sympy import parse_expr,symbols
import json
x,y,z,r,s,t = symbols("x y z r s t")

translation_parts={}

for i in range(227):
    unique_translations = set()
    sgnum = i+1    
    data = pd.read_html(f"https://www.cryst.ehu.es/cgi-bin/cryst/programs/nph-normsets?from=wycksets&gnum={sgnum}",match="Coset Representative",encoding='utf8')
    for frame in data:
        for datum in frame["Coset Representative"]:
            operations = [parse_expr(coord) for coord in datum.split(',')]
            translations = [op.evalf(subs={x: 0, y: 0, z: 0, r: 0, s: 0, t: 0}) for op in operations]
            unique_translations.add(tuple(translations))
    translation_parts[sgnum] = sorted(list(unique_translations))
    
for sgnum in translation_parts:
    for i,translation in enumerate(translation_parts[sgnum]):
        translation_parts[sgnum][i] = list(translation)
        for j,component in enumerate(translation_parts[sgnum][i]):
            translation_parts[sgnum][i][j] = float(component)
    
with open('translation_coset_representatives.json','w') as f:
    json.dump(translation_parts,f)



In [ ]:
# For each Bravais lattice, this generates P transpose, where P is the change of basis matrix defined in ITA 1.5.1.2, with "old basis" = primitive cell
# "new basis" = conventional cell as defined in the "aflow part 4" paper. 

import numpy as np
from sympy import symbols, cos, sin, Matrix, matrix2numpy, sqrt
a,b,c,alpha,beta,gamma = symbols('a b c alpha beta gamma')
import json

# using row vectors
primitivize_fractional_vector = {}

# mP
mP_cell = Matrix([
    [a,0,0],
    [0,b,0],
    [c*cos(beta),0,c*sin(beta)]
           ])

# mC
mC_cell = Matrix([
    [a/2,-b/2,0],
    [a/2,b/2,0],
    [c*cos(beta),0,c*sin(beta)]
])

primitivize_fractional_vector["mC"] = matrix2numpy(mP_cell@(mC_cell**-1),dtype=float).tolist()

# oP
oP_cell = Matrix([
    [a,0,0],
    [0,b,0],
    [0,0,c],
])

# oC
oC_cell = Matrix([
    [a/2,-b/2,0],
    [a/2,b/2,0],
    [0,0,c]
])

primitivize_fractional_vector["oC"]  = matrix2numpy(oP_cell@(oC_cell**-1),dtype=float).tolist()

# oA
oA_cell = Matrix([
    [a,0,0],
    [0,b/2,-c/2],
    [0,b/2,c/2]
])

primitivize_fractional_vector["oA"]  = matrix2numpy(oP_cell@(oA_cell**-1),dtype=float).tolist()

# oI
oI_cell = Matrix([
    [-a/2,b/2,c/2],
    [a/2,-b/2,c/2],
    [a/2,b/2,-c/2]
])

primitivize_fractional_vector["oI"]  = matrix2numpy(oP_cell@(oI_cell**-1),dtype=float).tolist()

# oF
oF_cell = Matrix([
    [0,b/2,c/2],
    [a/2,0,c/2],
    [a/2,b/2,0]
])

primitivize_fractional_vector["oF"]  = matrix2numpy(oP_cell@(oF_cell**-1),dtype=float).tolist()

# tP
tP_cell = Matrix([
    [a,0,0],
    [0,a,0],
    [0,0,c]
])

# tI
tI_cell = Matrix([
    [-a/2,a/2,c/2],
    [a/2,-a/2,c/2],
    [a/2,a/2,-c/2]
])

primitivize_fractional_vector["tI"]  = matrix2numpy(tP_cell@(tI_cell**-1),dtype=float).tolist()

# hP
hP_cell = Matrix([
    [a/2,-sqrt(3)*a/2,0],
    [a/2,sqrt(3)*a/2,0],
    [0,0,c]
])

# hR
hR_cell = Matrix([
    [a/2,-a/(2*sqrt(3)),c/3],
    [0,a/sqrt(3),c/3],
    [-a/2,-a/(2*sqrt(3)),c/3],
])

primitivize_fractional_vector["hR"]  = matrix2numpy(hP_cell@(hR_cell**-1),dtype=float).tolist()

# cP
cP_cell = Matrix([
    [a,0,0],
    [0,a,0],
    [0,0,a],
])

# cI
cI_cell = Matrix([
    [-a/2,a/2,a/2],
    [a/2,-a/2,a/2],
    [a/2,a/2,-a/2]
])

primitivize_fractional_vector["cI"]  = matrix2numpy(cP_cell@(cI_cell**-1),dtype=float).tolist()

# cF
cF_cell = Matrix([
    [0,a/2,a/2],
    [a/2,0,a/2],
    [a/2,a/2,0]
])

primitivize_fractional_vector["cF"]  = matrix2numpy(cP_cell@(cF_cell**-1),dtype=float).tolist()

with open("primitivize_fractional_vector.json","w") as f:
    json.dump(primitivize_fractional_vector,f)

In [ ]:
import json
from kim_tools import get_formal_bravais_lattice_from_space_group
import numpy as np

with open("primitivize_fractional_vector.json") as f:
    primitivize_fractional_vector = json.load(f)
with open("translation_parts.json") as f:
    translation_parts = json.load(f)
with open("translation_coset_representatives.json") as f:
    translation_coset_representatives = json.load(f)
    
possible_primitive_shifts = {}

for i in range(230):
    sgnum = str(i+1)
    possible_primitive_shifts_for_this_sg = []
    translation_parts_for_this_sg = translation_parts[sgnum]
    translation_coset_representatives_for_this_sg = translation_coset_representatives[sgnum]
    formal_pearson = get_formal_bravais_lattice_from_space_group(sgnum)
    # primitive cells aren't in the dict
    if formal_pearson in primitivize_fractional_vector:
        prim_matrix = np.asarray(primitivize_fractional_vector[formal_pearson])
    else:
        prim_matrix = np.eye(3)
        
    for translation_part in translation_parts_for_this_sg:
        translation_part_prim = translation_part@prim_matrix
        for coset_rep in translation_coset_representatives_for_this_sg:
            coset_rep_prim = coset_rep@prim_matrix
            primitive_shift = translation_part_prim+coset_rep_prim
            primitive_shift = [component%1 for component in primitive_shift]
            already_exists = False
            for existing_prim_shift in possible_primitive_shifts_for_this_sg:
                if np.allclose(primitive_shift,existing_prim_shift):
                    already_exists = True
                    break
            if already_exists:
                continue
            else:
                possible_primitive_shifts_for_this_sg.append(primitive_shift)
    possible_primitive_shifts[sgnum] = possible_primitive_shifts_for_this_sg

with open("possible_primitive_shifts.json","w") as f:
    json.dump(possible_primitive_shifts,f)
